# The `@property` Decorator in Python

**Overview:**
- In Python, the `@property` decorator allows you to define methods in a class that behave like attributes.
- This is often used for **encapsulation** (controlling access to, or validation of, an attribute).
- It helps keep your code clean and readable while providing the flexibility to add logic behind attribute access.

In this notebook, we will cover:
1. What is `@property`?
    2. Basic Usage
3. Getters, Setters, and Deleters
4. Practical Example
5. Pitfalls and Best Practices
6. When to Use `@property`

## 1. What is `@property`? <a id="what-is-property"></a>

- The `@property` decorator in Python transforms a method into a **getter** for a read-only attribute.
- It allows you to access a method like it is a regular attribute (`obj.attribute` instead of `obj.method()`).
- If needed, you can expand it with `@<property_name>.setter` and `@<property_name>.deleter` decorators to define **setter** and **deleter** behavior, respectively.

**Why is this helpful?**
- It provides a **clean** and **Pythonic** way to implement **encapsulation**.
- Allows you to **change internal implementation** (e.g., adding validation or extra computation) without changing the external interface for your class.

## 2. Basic Usage <a id="basic-usage"></a>

Below is a simple example showing how `@property` can be used to create a read-only property.

In [6]:
class Circle:
    def __init__(self, radius):
        self._radius = radius  # convention: underscore prefix to indicate "internal" usage

    @property
    def radius(self):
        """
        The radius property - read-only in this example.
        """
        return self._radius

# Usage
c = Circle(5)
print("Circle radius:", c.radius)

# Trying to set c.radius would fail because we haven't defined a setter
c.radius = 10


Circle radius: 5


AttributeError: property 'radius' of 'Circle' object has no setter

**Explanation**:
- We store the radius internally in `self._radius`.
- The `@property`-decorated `radius()` method turns into the *getter* for the attribute named `radius`.
- Attempting to assign a value to `c.radius` fails with `AttributeError` because no setter is defined.

---

## 3. Getters, Setters, and Deleters <a id="getters-setters-and-deleters"></a>

If you need to **allow setting** or **deleting** an attribute, you can define additional methods on top of the property:

- `@<property_name>.setter` for setting a value.
- `@<property_name>.deleter` for deleting the attribute.

**Syntax**:
```python
@property
def name(self):
    # getter logic

@name.setter
def name(self, value):
    # setter logic

@name.deleter
def name(self):
    # deleter logic

In [3]:
class Person:
    def __init__(self, name):
        self._name = name

    @property
    def name(self):
        """
        Getter for the 'name' property.
        """
        print("Getting name...")
        return self._name

    @name.setter
    def name(self, new_name):
        """
        Setter for the 'name' property.
        Here, we can add validation or additional logic.
        """
        print("Setting name...")
        if not new_name:
            raise ValueError("Name cannot be empty!")
        self._name = new_name

    @name.deleter
    def name(self):
        """
        Deleter for the 'name' property.
        This is optional and rarely used in practice,
        but can be handy for certain use cases.
        """
        print("Deleting name...")
        del self._name

# Usage:
p = Person("Alice")
print(p.name)           # Calls the getter
p.name = "Bob"          # Calls the setter
print(p.name)

# Let's try deleting the name:
del p.name             # Calls the deleter
# Now p._name no longer exists

Getting name...
Alice
Setting name...
Getting name...
Bob
Deleting name...


**Notes**:
- The `@<property_name>.setter` and `@<property_name>.deleter` decorator names must match the original `@property`-decorated method name.
- These decorators implicitly handle the logic for `__get__`, `__set__`, and `__delete__` in Python’s descriptor protocol, giving you a clean interface.

---

## 4. Practical Example <a id="practical-example"></a>

Let’s illustrate with a more realistic example. Imagine we want a **Temperature** class that stores the temperature in Celsius internally but provides:
    - A property `celsius` to get/set in Celsius.
- A property `fahrenheit` to get/set in Fahrenheit.

We can achieve this with two properties that reference the same internal state.

In [4]:
class Temperature:
    def __init__(self, celsius=0.0):
        # we'll store it internally as Celsius
        self._celsius = celsius

    @property
    def celsius(self):
        """Get or set temperature in Celsius."""
        return self._celsius

    @celsius.setter
    def celsius(self, value):
        print("Setting Celsius...")
        self._celsius = float(value)

    @property
    def fahrenheit(self):
        """Get or set temperature in Fahrenheit."""
        return (self._celsius * 9/5) + 32

    @fahrenheit.setter
    def fahrenheit(self, value):
        print("Setting Fahrenheit...")
        # convert fahrenheit to celsius
        self._celsius = (float(value) - 32) * 5/9

# Usage:
temp = Temperature()
print("Initial Celsius:", temp.celsius)       # 0.0
print("Initial Fahrenheit:", temp.fahrenheit) # 32.0

temp.celsius = 100
print("Fahrenheit after setting Celsius to 100:", temp.fahrenheit)

temp.fahrenheit = 32
print("Celsius after setting Fahrenheit to 32:", temp.celsius)

Initial Celsius: 0.0
Initial Fahrenheit: 32.0
Setting Celsius...
Fahrenheit after setting Celsius to 100: 212.0
Setting Fahrenheit...
Celsius after setting Fahrenheit to 32: 0.0


Here:
- Setting `temp.celsius` changes the internal `_celsius` value.
- Getting `temp.fahrenheit` calculates the value on the fly.
- Setting `temp.fahrenheit` updates the internal `_celsius` accordingly.

This is a powerful technique to keep your code DRY (Don’t Repeat Yourself) and keep a single source of truth (in this case, `_celsius`).

---

## 5. Pitfalls and Best Practices <a id="pitfalls-and-best-practices"></a>

1. **Don’t Overuse Properties**:
- If your logic is more than a few lines or you have complex computations, consider using regular methods to keep your code more explicit.
- Properties are best used for straightforward attribute-like access.

2. **Avoid Side Effects in Getters**:
- A property getter should ideally be simple and not cause unexpected behavior (e.g., network calls, writing to disk).
- Surprising side effects can confuse users of your class.

3. **Use an Internal Name Convention**:
- It’s standard in Python to store the underlying value in an attribute with a leading underscore (e.g., `self._attribute`) when using properties.
- This helps distinguish between the raw internal storage and the property accessor.

4. **Be Consistent**:
- If you define a property for one attribute in a class, consider doing it for all attributes that need logic or validation.
- Keep your approach uniform and documented.

5. **Don’t Break the User’s Expectations**:
- If setting a property has complex or *expensive* effects, document it well so users aren’t surprised by performance hits or side effects.

---

## 6. When to Use `@property` <a id="when-to-use-property"></a>

- **Encapsulation / Validation**: When you need to ensure valid data (e.g., `name` must not be empty, `age` must be positive).
- **Lazy Computation**: When the value can be computed on the fly and you’d like to retrieve it as if it’s an attribute.
- **Read-Only Attributes**: When an attribute should not be set externally but still be readable (e.g., a computed property).

**Key Benefit**:
- You can start by defining a public attribute (`self.name = ...`). If you later need to add validation or computations, you can refactor that attribute into a property **without** changing the external interface. Existing code using `obj.name` continues to work seamlessly.


In [5]:
"""
Previous code:


class Currency:
    def __init__(self, units, cents):
        self.units = units
        self.cents = cents

    # Currency implementation...


Now, say that your requirements change, and you decide to store the total number of cents instead of the units and cents. Removing .units and .cents from your public API to use something like .total_cents could break the code of more than one user.

"""

CENTS_PER_UNIT = 100

class Currency:
    def __init__(self, units, cents):
        self._total_cents = units * CENTS_PER_UNIT + cents

    @property
    def units(self):
        return self._total_cents // CENTS_PER_UNIT

    @units.setter
    def units(self, value):
        self._total_cents = self.cents + value * CENTS_PER_UNIT

    @property
    def cents(self):
        return self._total_cents % CENTS_PER_UNIT

    @cents.setter
    def cents(self, value):
        self._total_cents = self.units * CENTS_PER_UNIT + value

    # Currency implementation...


---

# Conclusion

- The `@property` decorator is a pythonic way of adding **getter/setter** behavior to attributes without changing how the attribute is accessed externally.
- It enhances code readability and maintains backward compatibility when behavior changes become necessary.
- Use it judiciously to keep your classes organized, validated, and easy to maintain.

**Further Reading**:
- Python official docs on the [property](https://docs.python.org/3/library/functions.html#property) built-in function.
- The [descriptor protocol](https://docs.python.org/3/howto/descriptor.html) for a deeper understanding of how properties work under the hood.

---

```python
# Feel free to experiment with additional examples here!
class Demo:
    def __init__(self):
        self._value = 0

    @property
    def value(self):
        """Example property."""
        return self._value

    @value.setter
    def value(self, v):
        """Example setter with validation."""
        if v < 0:
            raise ValueError("Value cannot be negative.")
        self._value = v

d = Demo()
d.value = 10
print("Demo value:", d.value)

# Let's try setting a negative value:
try:
    d.value = -5
    print("Caught error:", e)
except ValueError as e: